In [0]:
import requests
import json

In [0]:
api_key = "paste_here_you_key"

# Movie and credits
movie_id = 550
#url = "https://api.themoviedb.org/3/movie/"str(movie_id)"?api_key="+api_key+"&append_to_response=credits"

# Actor(Actress) and movie credits
person_id = 11701
url = "https://api.themoviedb.org/3/person/"+str(person_id)+"?api_key="+api_key+"&append_to_response=movie_credits"

# Find someone
person = "angelina jolie"
#url = "https://api.themoviedb.org/3/search/person?api_key="+api_key+"&query=\""+person+"\""


In [0]:
test = requests.get(url)
data = test.json()

In [38]:
#data["results"][0]["name"]
#data["results"][0]["id"]
data["movie_credits"]["cast"]

[{'adult': False,
  'backdrop_path': '/fAwYX0Fha35CaYYUJ06VTN3bEeh.jpg',
  'character': 'Jane Smith',
  'credit_id': '52fe4276c3a36847f80208c7',
  'genre_ids': [28, 35, 18, 53],
  'id': 787,
  'original_language': 'en',
  'original_title': 'Mr. & Mrs. Smith',
  'overview': "After five (or six) years of vanilla-wedded bliss, ordinary suburbanites John and Jane Smith are stuck in a huge rut. Unbeknownst to each other, they are both coolly lethal, highly-paid assassins working for rival organisations. When they discover they're each other's next target, their secret lives collide in a spicy, explosive mix of wicked comedy, pent-up passion, nonstop action and high-tech weaponry.",
  'popularity': 21.694,
  'poster_path': '/dqs5BmwSULtB28Kls3IB6khTQwp.jpg',
  'release_date': '2005-06-10',
  'title': 'Mr. & Mrs. Smith',
  'video': False,
  'vote_average': 6.6,
  'vote_count': 5689},
 {'adult': False,
  'backdrop_path': '/i76jVzAzlvg1BQAWmfwyrJV3an5.jpg',
  'character': 'Joan',
  'credit_id':